In [ ]:
from source.class_yolo_trainer import YOLOTrainer

yolo_trainer = YOLOTrainer
yolo_trainer = YOLOTrainer()

In [ ]:
### vou deixar anotado porque não sei se fiz gambiarra, 
""" no atributo image_folder para tarefas de classificação eu estou passando somente a pasta do dataset, sem indicar a pasta de imagens
"""
yolo_trainer.image_folder = r"emissoes_dataset_YOLO\images"
yolo_trainer.annotations_folder = r"emissoes_dataset_YOLO\labels"
yolo_trainer.yolo_Classes = ["emissao"]
yolo_trainer.dataset_path = "emissoes_YOLO"
yolo_trainer.task = "detect"
yolo_trainer.aug = True
yolo_trainer.n_aug = 10
yolo_trainer.odd = 0.12
yolo_trainer.test_percentual_divisor = 20

yolo_trainer.slicing()

In [ ]:
from source.modules.opt_yolo import objective
from ultralytics import YOLO
import optuna

model = YOLO(r"yolo11m.pt")

# model = r"YOLO_tools\yolo11m.pt"

images_dir = r'D:\Judson_projetos\Yolo_trainer\YOLO_tools\datasets\emissoes_YOLO\images'
labels_dir = r'D:\Judson_projetos\Yolo_trainer\YOLO_tools\datasets\emissoes_YOLO\labels'
data_yaml = r'D:\Judson_projetos\Yolo_trainer\YOLO_tools\datasets\emissoes_YOLO\dataset.yaml'

storage_url = "sqlite:///emissoes-fugitivas.db"
study = optuna.create_study(direction="maximize", storage=storage_url, study_name="opt-emissoes", load_if_exists=True)      ### study for F1 metric, thats why maximize
study.optimize(lambda trial: objective(trial, images_dir, labels_dir, model, data_yaml), n_trials=50)

In [ ]:
from ultralytics import YOLO
from ray import tune
import wandb
import yaml

#pip install -U ultralytics "ray[tune]"

#### YOLO é gambiarra e eu posso provar:
""" Para treinamentos de classificação com YOLO, você deve indicar o dir com o dataset
que deve estar especificado dentro de uma pasta chamada 'datasets'. No entando, para detecção
o YOLO é diferente. Você precisa indicar o caminho do arquivo 'dataset.yaml' para que ele 
possa encontrar o dataset e realizar o treinamento. É a mesma função, de uma mesma lib,
mas os caras fizeram de forma que o mesmo argumento recebe duas entradas completamente 
diferentes a depender do treinamento que você vai fazer.
"""

# Carregar configurações de um arquivo YAML
with open('../hyper_yolo.yaml', 'r') as file:
    config = yaml.safe_load(file)

train_config = config['train']
aug_config = config['train']['augmentation']

model = YOLO(r"yolo11m.pt")

wandb.init(project="YOLO-Tuning", entity="your-entity")

results = model.tune(
    data = "datasets/emissoes_YOLO/dataset.yaml",
    device = "cuda",

    ### training configs. Checkout hyper_yolo.yaml for details
    imgsz = train_config['imgsz'],
    batch = train_config['batch'],
    epochs = train_config['epochs'],
    optimizer = train_config['optimizer'],

    warmup_epochs = train_config['warmup_epochs'],
    warmup_momentum = train_config['warmup_momentum'],
    warmup_bias_lr = train_config['warmup_bias_lr'],

    momentum = train_config['momentum'],
    lr0 = train_config['lr0'],
    lrf = train_config['lrf'],
    weight_decay = train_config['weight_decay'],

    space={
        "lr0": tune.uniform(1e-5, 1e-1),
        "lrf": tune.uniform(1e-5, 1e-2),
        "weight_decay": tune.uniform(1e-3, 1e-2),
        "momentum": tune.uniform(0.8, 0.95),
        "warmup_epochs": tune.randint(1, 5),
        "warmup_momentum": tune.uniform(0.4, 0.8),
        "warmup_bias_lr": tune.uniform(1e-5, 1e-1),
        "epochs": tune.randint(50, 200),
        "optimizer": tune.choice(['SGD', "AdamW"]),
        "imgsz": tune.choice([360, 480, 640, 1024]),
        "batch": tune.randint(8, 48),
        },

    use_ray=True,

)

In [1]:
from ultralytics import YOLO
from ray import tune
import hashlib
import wandb
import ray
import os

def train_yolo_with_ray(config):    # Função objetivo para ajuste

    model = YOLO("yolo11m.pt")  # Inicializar o modelo dentro da função

    wandb.init(    # Configurar W&B dentro de cada execução
        project="YOLO-Tuning",
        config=config
    )

    results = model.tune(        ### training configs. Checkout config for details
        data = r"D:\Judson_projetos\Yolo_trainer\YOLO_tools\datasets\emissoes_YOLO\dataset.yaml",
        device = "cuda",

        imgsz = config['imgsz'],
        batch = config['batch'],
        epochs = config['epochs'],
        optimizer = config['optimizer'],

        warmup_epochs = config['warmup_epochs'],
        warmup_momentum = config['warmup_momentum'],
        warmup_bias_lr = config['warmup_bias_lr'],

        momentum = config['momentum'],
        lr0 = config['lr0'],
        lrf = config['lrf'],
        weight_decay = config['weight_decay'],
        )

    wandb.log({    # Logar métricas ao W&B
        # "train_loss": results.box_loss[-1],
        # "val_loss": results.val_loss[-1],
        # "mAP50": results.metrics["mAP50"],
        "val_loss": results.metrics.get('val_loss', None),
        "cls_loss": results.metrics.get('cls_loss', None),
        "box_loss": results.metrics.get('box_loss', None),         # A perda (loss) ainda está disponível
        "mAP_50_95": results.metrics.get('mAP_50_95', None),  # Usando .get() para evitar erro caso a métrica não exista        # Para acessar o mAP, você pode acessar 'metrics' no resultado
        "f1_score": results.metrics.get('F1', None),        # Para acessar o F1-Score
    })
    wandb.finish()

space={     # Configurar o espaço de busca
    "lr0": tune.uniform(1e-5, 1e-1),
    "lrf": tune.uniform(1e-5, 1e-2),
    "weight_decay": tune.uniform(1e-3, 1e-2),
    "momentum": tune.uniform(0.8, 0.95),
    "warmup_epochs": tune.randint(1, 5),
    "warmup_momentum": tune.uniform(0.4, 0.8),
    "warmup_bias_lr": tune.uniform(1e-5, 1e-1),
    "epochs": tune.randint(50, 200),
    "optimizer": tune.choice(['SGD', "AdamW"]),
    "imgsz": tune.choice([360, 480, 640, 1024]),
    "batch": tune.randint(8, 48),
    }

def shorten_trial_dirname(trial):  # Ajustando para receber o objeto trial
    # Gerar um hash curto do trial_id para garantir que o nome do diretório seja único e curto
    return hashlib.md5(trial.trial_id.encode()).hexdigest()[:8]

ray.init(_temp_dir=r"D:\Judson_projetos\Yolo_trainer\YOLO_tools\ray_sessions")

analysis = tune.run(    # Iniciar o Ray Tune
    train_yolo_with_ray,
    config=space,
    num_samples=50,  # Número de execuções
    trial_dirname_creator=shorten_trial_dirname,
    resources_per_trial={"cpu": 4, "gpu": 1},  # Ajuste conforme disponível
    storage_path=r"D:\Judson_projetos\Yolo_trainer\YOLO_tools\ray_sessions"  # Definindo o caminho para as sessões do Ray
)

2025-01-06 16:55:29,893	INFO worker.py:1821 -- Started a local Ray instance.
2025-01-06 16:55:32,075	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(train_yolo_with_ray pid=6148) Downloading https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt to 'yolo11m.pt'...


  0%|          | 0.00/38.8M [00:00<?, ?B/s]
  4%|▍         | 1.62M/38.8M [00:00<00:02, 13.9MB/s]
 16%|█▌        | 6.25M/38.8M [00:00<00:01, 31.0MB/s]
 26%|██▌       | 10.0M/38.8M [00:00<00:00, 34.2MB/s]
 34%|███▍      | 13.4M/38.8M [00:00<00:00, 33.0MB/s]
 43%|████▎     | 16.6M/38.8M [00:00<00:00, 29.5MB/s]
 51%|█████     | 19.6M/38.8M [00:00<00:00, 26.9MB/s]
 58%|█████▊    | 22.4M/38.8M [00:00<00:00, 22.7MB/s]
 64%|██████▍   | 24.8M/38.8M [00:01<00:01, 11.9MB/s]
 68%|██████▊   | 26.5M/38.8M [00:01<00:01, 9.30MB/s]
 72%|███████▏  | 27.9M/38.8M [00:01<00:01, 8.41MB/s]
 75%|███████▍  | 29.0M/38.8M [00:02<00:01, 7.80MB/s]
 77%|███████▋  | 30.0M/38.8M [00:02<00:01, 7.01MB/s]
 80%|███████▉  | 30.9M/38.8M [00:02<00:01, 6.95MB/s]
 82%|████████▏ | 31.6M/38.8M [00:02<00:01, 6.54MB/s]
 83%|████████▎ | 32.4M/38.8M [00:02<00:01, 6.30MB/s]
 85%|████████▌ | 33.1M/38.8M [00:02<00:00, 6.28MB/s]
 87%|████████▋ | 33.8M/38.8M [00:03<00:00, 6.25MB/s]
 89%|████████▊ | 34.4M/38.8M [00:03<00:00, 5.53MB/s]
 9

(train_yolo_with_ray pid=6148) Tuner: Initialized Tuner instance with 'tune_dir=d:\Judson_projetos\Yolo_trainer\runs\detect\tune3'
(train_yolo_with_ray pid=6148) Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
(train_yolo_with_ray pid=6148) Tuner: Starting iteration 1/10 with hyperparameters: {'lr0': 0.0494, 'lrf': 0.00841, 'momentum': 0.92452, 'weight_decay': 0.001, 'warmup_epochs': 2, 'warmup_momentum': 0.40074, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}
(train_yolo_with_ray pid=6148) Ultralytics 8.3.58 🚀 Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
(train_yolo_with_ray pid=6148) engine\trainer: task=detect, mode=train, model=yolo11m.pt, data=D:\Judson_projetos\Yolo_trainer\YOLO_tools\datasets\emissoes_YOLO\d

  0%|          | 0.00/5.35M [00:00<?, ?B/s]
  9%|▉         | 512k/5.35M [00:00<00:01, 4.93MB/s]
 26%|██▌       | 1.38M/5.35M [00:00<00:00, 6.97MB/s]
 44%|████▍     | 2.38M/5.35M [00:00<00:00, 8.25MB/s]
 72%|███████▏  | 3.88M/5.35M [00:00<00:00, 10.9MB/s]
100%|██████████| 5.35M/5.35M [00:00<00:00, 11.1MB/s]


...

(train_yolo_with_ray pid=6148)                    all        726        836    0.00113     0.0191    0.00064   0.000121


  0%|          | 0/134 [00:00<?, ?it/s] 


(train_yolo_with_ray pid=6148) 
(train_yolo_with_ray pid=6148)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      3/185      6.68G        2.6      3.096      2.756         57        480:   1%|          | 1/134 [00:00<01:12,  1.84it/s]
      3/185      6.71G      2.724      3.029      2.784         61        480:   1%|▏         | 2/134 [00:00<00:44,  2.97it/s]
      3/185      6.71G      2.707      3.059      2.749         45        480:   2%|▏         | 3/134 [00:00<00:39,  3.35it/s]
      3/185      6.71G      2.768      3.148      2.783         52        480:   3%|▎         | 4/134 [00:01<00:33,  3.84it/s]
      3/185      6.71G      2.778      3.164      2.785         42        480:   4%|▎         | 5/134 [00:01<00:30,  4.20it/s]
      3/185      6.71G      2.787      3.199      2.792         56        480:   4%|▍         | 6/134 [00:01<00:30,  4.17it/s]
      3/185      6.71G      2.778      3.201      2.787         55        480:   5%|▌         | 7/134 [00:01<00:28,  4.46it/s]
      3/185      6.71G      2.765      3.197      2.778         46        480:   6%|▌         | 8/134 [00:02<00

(train_yolo_with_ray pid=6148)                    all        726        836     0.0103        0.1    0.00964    0.00182


In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
import torch
import torchvision
print(f"PyTorch version: {torch.__version__}")
print(f"TorchVision version: {torchvision.__version__}")
print(torchvision.ops.nms.__module__)



In [ ]:
from ultralytics import YOLO
import yaml

#### YOLO é gambiarra e eu posso provar:
""" Para treinamentos de classificação com YOLO, você deve indicar o dir com o dataset
que deve estar especificado dentro de uma pasta chamada 'datasets'. No entando, para detecção
o YOLO é diferente. Você precisa indicar o caminho do arquivo 'dataset.yaml' para que ele 
possa encontrar o dataset e realizar o treinamento. É a mesma função, de uma mesma lib,
mas os caras fizeram de forma que o mesmo argumento recebe duas entradas completamente 
diferentes a depender do treinamento que você vai fazer.
"""

# Carregar configurações de um arquivo YAML
with open('../hyper_yolo.yaml', 'r') as file:
    config = yaml.safe_load(file)

train_config = config['train']
aug_config = config['train']['augmentation']

model = YOLO("runs\\detect\\train52\\weights\\best.pt")

results = model.train(
    data = "datasets/emissoes_YOLO/dataset.yaml",
    device = "cuda",

    ### training configs. Checkout hyper_yolo.yaml for details
    imgsz = train_config['imgsz'],
    batch = train_config['batch'],
    weight_decay = train_config['weight_decay'],

    warmup_epochs = train_config['warmup_epochs'],
    warmup_momentum = train_config['warmup_momentum'],
    warmup_bias_lr = train_config['warmup_bias_lr'],

    epochs = train_config['epochs'],
    momentum = train_config['momentum'],
    lr0 = train_config['lr0'],
    lrf = train_config['lrf'],
    optimizer = train_config['optimizer'],

    ### if you r willing to use yolo aug, uncomment section bellow
    ### not recomended if ur data is already aug though
    ### augment args
    # hsv_h = aug_config['hsv_h'],
    # hsv_s = aug_config['hsv_s'],
    # hsv_v = aug_config['hsv_v'],
    # degrees = aug_config['degrees'],
    # translate = aug_config['translate'],
    # scale = aug_config['scale'],
    # shear = aug_config['shear'],
    # perspective = aug_config['perspective'],
    # flipud = aug_config['flipud'],
    # fliplr = aug_config['fliplr'],
    # mosaic = aug_config['mosaic'],
    # mixup = aug_config['mixup'],
    # copy_paste = aug_config['copy_paste'],
    # auto_augment = aug_config['auto_augment'],
)

In [ ]:
from ultralytics import YOLO

model = YOLO("runs\\detect\\train52\\weights\\best.pt")

model.info()

In [ ]:
from ultralytics import YOLO

train = 'train52'
model = YOLO(f"runs/detect/{train}/weights/best.pt")
model.predict("../RUIM 23-06-22 (1).mp4", save=True, conf=0.4, device="cuda", save_txt=False, save_conf=True, save_crop=False)

In [ ]:
from utils.check_file import check_file

check_file("datasets\\emissoes_YOLO\\dataset.yaml")

In [ ]:
from utils.torch_is_available import torch_is_available

torch_is_available()

In [ ]:
from utils.cvat_dataset import cvat_dataset

cvat_dataset("../emissoes_dataset_CVAT", "emissoes_dataset_YOLO")

In [ ]:
from utils.remover_labels_vazios import processar_dataset

processar_dataset("emissoes_dataset_YOLO")